# Glider data analysis for Phycoglider II

We start with a list of phycoglider missions by using the search box on the observations portal missions page https://observations.voiceoftheocean.org/missions

Those are: 

- SEA077 M44
- SEA045 M100
- SEA077 M46

In [ ]:
import xarray as xr
import numpy as np
import glidertools
import glidertools as gt
import cmocean.cm as cmo
import matplotlib.pyplot as plt
import requests
from pathlib import Path
data_dir = Path("data/raw_voto")
if not data_dir.exists():
    data_dir.mkdir(parents=True)

For each mission, we will download the glider data and ADCP data, load that data into [GliderTools](https://github.com/GliderToolsCommunity/GliderTools), and apply some of the GliderTools BGC corrections

In [ ]:
deployment_id = "SEA045_M100"
#deployment_id = "SEA077_M44"
#deployment_id = "SEA077_M46"


In [ ]:
url_glider = f"https://erddap.observations.voiceoftheocean.org/erddap/files/delayed_{deployment_id}/mission_timeseries.nc"
url_ad2cp = f"https://erddap.observations.voiceoftheocean.org/erddap/files/gliderad2cp_files/{deployment_id}_adcp_proc.nc"
data_file = data_dir / f"{deployment_id}.nc"
adcp_file = data_dir / f"{deployment_id}_adcp_proc.nc"

if not data_file.exists():
    response_glider = requests.get(url_glider)
    if response.status_code == 200:
        with open(data_file, "wb") as f:
            f.write(response_glider.content)
    else:
        print(f"failed for {url_glider}")
if not adcp_file.exists():
    response_ad2cp = requests.get(url_ad2cp)
    if response_ad2cp.status_code == 200:
    
        with open(adcp_file, "wb") as f:
            f.write(response_ad2cp.content)
    else:
        print(f"failed for {url_ad2cp}")

In [ ]:
ds = xr.open_dataset(data_file)


In [ ]:
dsa = ds.sel({"time":slice(ds.time[1000], ds.time[10000])})
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
mappable = ax.scatter(dsa.time, dsa.depth, c=dsa.temperature, cmap=cmo.thermal)
ax.invert_yaxis()
fig.colorbar(ax=ax,mappable=mappable, label='Temperature (C)')

In [ ]:
dsa = ds.sel({"time":slice(ds.time[10000], ds.time[30000])})
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
mappable = ax.scatter(dsa.time, dsa.depth, c=dsa.phycocyanin, cmap=cmo.algae, vmin=0, vmax=10, s=1)
ax.invert_yaxis()
fig.colorbar(ax=ax,mappable=mappable, label='Phycocyanin')

### ADCP display

In [ ]:
adcp = xr.open_dataset(adcp_file)
adcp = adcp.sel({"profile_index":slice(0, adcp.profile_index[np.nanargmax(adcp.time)])})

In [ ]:
variables = ["velocity_N_no_reference", "velocity_N_DAC_reference", "velocity_N_DAC_reference_sb_corrected"]
titles = ["No referencing", "DAC referencing", "DAC referencing, bias correction"]
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
mappable = ax.pcolormesh(adcp.time[:-1], adcp.depth, adcp["velocity_N_no_reference"][:, :-1], cmap=cmo.balance, vmin=-0.2, vmax=0.2)
ax.invert_yaxis()
fig.colorbar(ax=ax,mappable=mappable, label='Northward velocity (m/s)')
ax.set(ylabel='Depth (m)', title="ADCP velocity unreferenced")

When referenced to DAC, we have strong, directionally dependant currents. Indicates some issues with the compass in the collection or processing. We will investigate

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
mappable = ax.pcolormesh(adcp.time[:-1], adcp.depth, adcp["velocity_N_DAC_reference"][:, :-1], cmap=cmo.balance, vmin=-0.6, vmax=0.6)
ax.invert_yaxis()
fig.colorbar(ax=ax,mappable=mappable, label='Northward velocity (m/s)')
ax.set(ylabel='Depth (m)', title="ADCP velocity referenced to DAC")

### Load into GliderTools

In [ ]:
df = ds.to_dataframe()

In [ ]:
dat = gt.load.voto_seaexplorer_nc("data/raw_voto/SEA045_M100.nc")
depth = dat.depth
dives = dat.dive_num
lats = dat.latitude
lons = dat.longitude
time = dat.time
pres = dat.pressure
temp = dat.temperature
salt = dat.salinity
flour = dat.chlorophyll
phyc = dat.phycocyanin
backscat = dat.backscatter
# name coordinates for quicker plotting
x = dat.dives
y = dat.depth

In [ ]:
gt.plot(x, y, salt, cmap=cmo.haline, robust=True)
plt.title('Original Data')
plt.show()

In [ ]:
flr_qc, quench_layer = gt.optics.quenching_correction(
    dat.chlorophyll, backscat, dives, depth, time, lats, lons,
    sunrise_sunset_offset=0, night_day_group=True)

fig, ax = plt.subplots(3, 1, figsize=[12, 9], sharex=True, dpi=90)
gt.plot(x, y, flour, cmap=cmo.delta, ax=ax[0], robust=True)
gt.plot(x, y, flr_qc, cmap=cmo.delta, ax=ax[1], robust=True)
gt.plot(x, y, quench_layer, cmap='RdBu_r', ax=ax[2], vmin=-.5, vmax=2)

[a.set_xlabel('') for a in ax]
[a.set_ylim(40, 0) for a in ax]
ax[0].set_title('Uncorrected Fluorescence')
ax[1].set_title('Quenching Corrected Fluorescence')
ax[2].set_title('Quenching Layer')

plt.show()